# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Load the environment variable `PRICE_DATA`
PRICE_DATA = os.getenv('PRICE_DATA')

# Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# DASK + PANDAS
# Read files
ddf = dd.read_parquet(parquet_files).set_index("ticker")

def add_features(df):
    df = df.sort_values("Date")
    df['Close_lag_1'] = df['Close'].shift(1)
    df['Adj_Close_lag_1'] = df['Adj Close'].shift(1)
    df['returns'] = (df['Close']/df['Close_lag_1'] - 1)
    df['hi_lo_range'] = df['High'] - df['Low']
    return df

dd_feat = ddf.map_partitions(lambda df: df.groupby('ticker', group_keys=False).apply(add_features))

# Convert the Dask data frame to a pandas data frame
pd_feat = dd_feat.compute()
# Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`
pd_feat= pd_feat.groupby('ticker').apply(
    lambda df: df.assign(mov_avg = df['returns'].rolling(10).mean())
)
# remove extra index created
pd_feat.reset_index(drop=True, inplace=True)
pd_feat['mov_avg'].sum() # 26373.228049293215 sorted by date
                         # 31119.772070707193 not sorted by date

#pd_feat
#pd_feat.compute()
#type(pd_feat.compute())
#len(pd_feat) # 359088
#len(pd_feat.columns) #13


26373.228049293215

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

No, we could have done it with Dask
+ Would it have been better to do it in Dask? Why?

In Dask, there is bigger overhead as it does paralelization and prepares the processes, but then it calculates it faster.

It is a trade off between overhead time vs calculation saved time.

This size, likely it would have been better to use Dask. 

See below code to do the moving average feature. In my computer, doing it with Pandas takes 16 seconds, doing it with Dask shows 0.0 seconds if not computed. 

It doesn't surprise me, in the past I was encountering 100x performance improvements with Numpy arrays instead of Pandas dataframes.

(1 pt)

In [ ]:
# DASK only
# Read files
dd_df = dd.read_parquet(parquet_files).set_index("ticker")

def dd_add_features(df):
    df = df.sort_values("Date")
    df['Close_lag_1'] = df['Close'].shift(1)
    df['Adj_Close_lag_1'] = df['Adj Close'].shift(1)
    df['returns'] = (df['Close']/df['Close_lag_1'] - 1)
    df['hi_lo_range'] = df['High'] - df['Low']
    df['mov_avg'] = df['returns'].rolling(10).mean()
    return df

dd_df_feat = dd_df.map_partitions(lambda df: df.groupby('ticker', group_keys=False).apply(dd_add_features))

dd_df_feat['mov_avg'].sum().compute()       # 26373.228049293204 sorted by date
                                            # 26755.444160583316 not sorted by date

#dd_feat_mov_avg.reset_index().compute()

# https://docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.reset_index.html
# Note that unlike in pandas, the reset index for a Dask DataFrame will not be monotonically increasing from 0. 
# Instead, it will restart at 0 for each partition (e.g. index1 = [0, ..., 10], index2 = [0, ...]). 
# This is due to the inability to statically know the full length of the index.

26373.228049293204

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.